In [1]:
import os

import dotenv
dotenv.load_dotenv()

MPAN: str = os.environ["ELECTRICITY_MPAN"]
SERIAL: str = os.environ["ELECTRICITY_SERIAL"]

In [2]:
from octopus import OctopusEnergyAPIClient

with OctopusEnergyAPIClient() as client:
    meter = client.get_electricity_meter_point_v1(MPAN)
    gsp = meter["gsp"]

    print("Tariffs:")
    for product in client.list_products_v1():
        if product["is_business"] or product["is_prepay"] or product["is_restricted"]:
            continue
        product = client.retrieve_product_v1(product["code"])
        if "Tracker" in product["display_name"]:
            print(product)
        if "Outgoing" in product["display_name"] or "Import" in product["display_name"] or "Export" in product["display_name"]:
            continue
        if gsp not in product["single_register_electricity_tariffs"]:
            continue
        if "direct_debit_monthly" not in product["single_register_electricity_tariffs"][gsp]:
            continue
        print(f"{product['display_name']} ({product['code']}):")
        values = product["single_register_electricity_tariffs"][gsp]["direct_debit_monthly"]
        print(f"  {values['code']}")
        print(f"  Standing charge: {values['standing_charge_inc_vat']} p")
        print(f"  Unit rate: {values['standard_unit_rate_inc_vat']} p")

Tariffs:
Agile Octopus (AGILE-24-04-03):
  E-1R-AGILE-24-04-03-G
  Standing charge: 49.119 p
  Unit rate: 32.571 p
Agile Octopus (AGILE-BB-24-04-03):
  E-1R-AGILE-BB-24-04-03-G
  Standing charge: 49.119 p
  Unit rate: 32.571 p
Co-op 12M Fixed (COOP-FIX-12M-24-08-28):
  E-1R-COOP-FIX-12M-24-08-28-G
  Standing charge: 49.119 p
  Unit rate: 24.998085 p
Cosy Octopus (COSY-22-12-08):
  E-1R-COSY-22-12-08-G
  Standing charge: 49.119 p
  Unit rate: 33.171285 p
Cosy Octopus (COSY-BB-23-02-07):
  E-1R-COSY-BB-23-02-07-G
  Standing charge: 49.119 p
  Unit rate: 33.171285 p
Co-op Community Power (CP-12M-24-08-28):
  E-1R-CP-12M-24-08-28-G
  Standing charge: 49.119 p
  Unit rate: 25.55364 p
Octopus Go (GO-VAR-22-10-14):
  E-1R-GO-VAR-22-10-14-G
  Standing charge: 49.119 p
  Unit rate: 23.8959 p
Octopus Go (GO-VAR-BB-23-02-07):
  E-1R-GO-VAR-BB-23-02-07-G
  Standing charge: 49.119 p
  Unit rate: 23.8959 p
Intelligent Octopus Go (INTELLI-BB-VAR-23-03-01):
  E-1R-INTELLI-BB-VAR-23-03-01-G
  Standing 

In [3]:
from datetime import date, timedelta

import pandas as pd

with OctopusEnergyAPIClient() as client:
    df = pd.DataFrame.from_records(client.list_electricity_meter_consumption_v1(MPAN, SERIAL, period_from=date.today() - timedelta(days=7)))
df["interval_start"] = pd.to_datetime(df["interval_start"])
df["interval_end"] = pd.to_datetime(df["interval_end"])
df["time"] = df["interval_start"] + (df["interval_end"] - df["interval_start"]) / 2
df

,consumption,interval_start,interval_end,time
0,0.201,2024-09-07 00:30:00+01:00,2024-09-07 01:00:00+01:00,2024-09-07 00:45:00+01:00
1,0.347,2024-09-07 00:00:00+01:00,2024-09-07 00:30:00+01:00,2024-09-07 00:15:00+01:00
2,0.382,2024-09-06 23:30:00+01:00,2024-09-07 00:00:00+01:00,2024-09-06 23:45:00+01:00
3,0.411,2024-09-06 23:00:00+01:00,2024-09-06 23:30:00+01:00,2024-09-06 23:15:00+01:00
4,0.439,2024-09-06 22:30:00+01:00,2024-09-06 23:00:00+01:00,2024-09-06 22:45:00+01:00
...,...,...,...,...
333,0.095,2024-08-31 02:00:00+01:00,2024-08-31 02:30:00+01:00,2024-08-31 02:15:00+01:00
334,0.126,2024-08-31 01:30:00+01:00,2024-08-31 02:00:00+01:00,2024-08-31 01:45:00+01:00
335,0.288,2024-08-31 01:00:00+01:00,2024-08-31 01:30:00+01:00,2024-08-31 01:15:00+01:00
336,0.425,2024-08-31 00:30:00+01:00,2024-08-31 01:00:00+01:00,2024-08-31 00:45:00+01:00


In [4]:
import plotly.express as px

fig = px.histogram(df, x="time", y="consumption", title=f"Electricity consumption for {MPAN}")
fig.update_layout(bargap=0.1)
fig.show()